In [2]:
import numpy as np
from numba import jit, vectorize, float64, int64

In [3]:
import stahmctestt as stt

## Derivation of gradient
#### Combine real data    
$U(\theta)=log(p(\theta|D)+log(p(\theta))$

Assume here the prior for $\theta$ $N(0,diag(2^2,4^2,5^2,0.1^2,0.1^2))$

$log(p(\theta|D_i)=y_ilogp+(1-y_i)log(1-p)=y_ilog(p/1-p)+log(1-p)=y_iX_i'\theta+log(\frac{1}{1+exp(X_i'\theta)})$   

$\frac{\partial log(p(\theta|D_i)}{\partial \theta}=y_iX_i-\frac{exp(X_i'\theta)}{1+exp(X_i'\theta)}X_i$
     
$log(p(\theta))=-1/2 \theta' V^{-1} \theta+C$   

$\frac{\partial log(p(\theta))}{\partial \theta}=- V^{-1} \theta$   

In [4]:
###simulate data
def sim_log(m,n,theta):
    """This function outputs a simulated logistics regression dataset X and y. 
       X is a matrix of size m by n. Every entry of X is i.i.d N(0,1).
    Args:
        m: the number of rows in X
        n: the number of columns in X
        beta: a n by 1 vector, the true beta in the model
    """
    X=np.random.normal(loc=0,scale=1,size=[m,n])
    y=np.random.binomial(n=1,p=1/(1+np.exp(-(X @ theta))))
    return X,y

In [ ]:
theta=np.array([1,2,-5,0,0])
X,y=sim_log(20000,5,theta)

In [5]:
#gradient
@jit(cache=True)
def gradlogistic(theta,X,y):  
    """
    theta:p.1-1d array
    X:n.p-2d array
    y n.1-1d array"""
    drll=np.diag(y-1/(1+np.exp(-X@theta)))@X
    drpri=-np.linalg.solve(np.diag([16,16,16,16,16]),theta)
    return -drpri,-drll.mean(axis=0)

In [9]:
theta0=np.zeros(5)
M=C=np.identity(5)
epsilon=.001
size=200
out=stt.hmc_nbvec(gradlogistic,X,y,theta0,M,C,epsilon,size,iter=30000)

In [10]:
out[:,:5].mean(0)

array([ 1.17074869e+00,  2.30513234e+00, -5.68114730e+00, -2.17634100e-03,
        3.23174946e-03])